In [54]:
import pandas as pd
from pathlib import Path

In [55]:
#create paths to csv files
school_data = Path("Resources/schools_complete.csv")
student_data = Path("Resources/students_complete.csv")

In [220]:
# read csv files
school_df = pd.read_csv(school_data)
student_df = pd.read_csv(student_data)
#school_df

In [57]:
# combine datasets
combined_df = pd.merge(student_df, school_df, how="left", on=["school_name",
                                                             "school_name"])
combined_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [ ]:
#DISTRICT SUMMARY

In [58]:
# Total number of unique school
school_count = combined_df['school_name'].nunique()
school_count

15

In [59]:
# Total number of students
student_count = combined_df['student_name'].count()
student_count

39170

In [60]:
# total budget
total_budget = school_df['budget'].sum()
total_budget

24649428

In [61]:
# Average Math Score
avg_math_score = combined_df['math_score'].mean()
avg_math_score

78.98537145774827

In [62]:
#Average Reading Score
avg_reading_score = combined_df['reading_score'].mean()
avg_reading_score

81.87784018381414

In [63]:
# % of students who passed math
passing_math_count = combined_df[(combined_df["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage

74.9808526933878

In [64]:
# % of students who passed reading
passing_reading_count = combined_df[(combined_df["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100
passing_reading_percentage

85.80546336482001

In [65]:
# overall % of students who passed math and reading
passing_math_reading_count = combined_df[
    (combined_df["math_score"] >= 70) & (combined_df["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate

65.17232575950983

In [69]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = pd.DataFrame({'Total Schools': [school_count], 'Total Students': [student_count], 
                                'Total Budget': [total_budget], 'Average Math Score': [avg_math_score],
                                'Average Reading Score': [avg_reading_score], '% of Students who Passed Math' : [passing_math_percentage],
                                '% of Students who Passed Reading' : [passing_reading_percentage],
                                'Overall Passing Rate': [overall_passing_rate]})

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)


# Display the DataFrame
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% of Students who Passed Math,% of Students who Passed Reading,Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


In [70]:
# SCHOOL SUMMARY

In [153]:
#school names
#school_name = combined_df.set_index('school_name')['type'].unique()

In [375]:
# selecting all school types
#school_types = combined_df.drop_duplicates('school_name').set_index('school_name')['type']
school_types = combined_df.groupby('school_name')['type'].first()
#school_types

In [263]:
# calculate total number of students per school
per_school_counts = combined_df.groupby('school_name')['size'].first()
#per_school_counts

In [286]:
# Calculate the total school budget and per capita spending per school
per_school_budget = combined_df.groupby('school_name')['budget'].first()
#per_school_capita = per_school_budget / per_school_counts
per_school_capita = per_school_budget / combined_df.groupby('school_name')['student_name'].count()
#per_school_budget
#per_school_capita


In [157]:
# Calculate the average test scores per school
per_school_math = combined_df.groupby('school_name').agg(
    Avg_Math_Score=('math_score', 'mean'))
per_school_reading = combined_df.groupby('school_name').agg(
    Avg_Reading_Score=('reading_score', 'mean'))
#per_school_math
#per_school_reading

In [269]:
# Calculate the number of students per school with math scores of 70 or higher
students_passing_math = combined_df[combined_df['math_score'] >= 70]
school_students_passing_math = students_passing_math.groupby('school_name')['Student ID'].count()
#school_students_passing_math

In [160]:
# Calculate the number of students per school with reading scores of 70 or higher
students_passing_reading = combined_df[combined_df['reading_score'] >= 70]
school_students_passing_reading = students_passing_reading.groupby('school_name')['Student ID'].count()
#school_students_passing_reading

In [161]:
#code from sample
students_passing_math_and_reading = combined_df[
    (combined_df["reading_score"] >= 70) & (combined_df["math_score"] >= 70)
]
school_students_passing_math_and_reading = students_passing_math_and_reading.groupby(["school_name"]).size()

In [287]:
per_school_passing_math = school_students_passing_math / per_school_counts * 100
per_school_passing_reading = school_students_passing_reading / per_school_counts * 100
overall_passing_rate = school_students_passing_math_and_reading / per_school_counts * 100
#per_school_passing_math

In [278]:
# Create DataFrame
#per_school_summary = pd.DataFrame({'School Name': [school_name], 'School Types': [school_types], 
                                #'Total Students': [per_school_counts], 'Total School Budget': [per_school_budget],
                                #'Per Student Budget': [per_school_capita], 'Average Math Score' : [per_school_math],
                                #'Average Reading Score' : [per_school_reading],
                                #'% Passing Math': [per_school_passing_math],'% Passing Reading': [per_school_passing_reading],'% Overall Passing': [overall_passing_rate]})

#school_name_df = pd.DataFrame(school_types)
per_school_summary = pd.DataFrame({
    'School Name': school_types.index,
    'School Type': school_types.values,
    'Total Students': per_school_counts.values,
    'Total School Budget': per_school_budget.values,
    'Per Student Budget': per_school_capita.values,
    'Average Math Score': school_students_passing_math.values,
    'Average Reading Score': school_students_passing_reading.values,
    '% Passing Math': per_school_passing_math.values,
    '% Passing Reading' : per_school_passing_reading.values,
    '% Overall Passing' : overall_passing_rate.values

})
# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the DataFrame
per_school_summary

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Bailey High School,District,4976,"$3,124,928.00",$628.00,3318,4077,66.680064,81.933280,54.642283
1,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,1749,1803,94.133477,97.039828,91.334769
2,Figueroa High School,District,2949,"$1,884,411.00",$639.00,1946,2381,65.988471,80.739234,53.204476
3,Ford High School,District,2739,"$1,763,916.00",$644.00,1871,2172,68.309602,79.299014,54.289887
4,Griffin High School,Charter,1468,"$917,500.00",$625.00,1371,1426,93.392371,97.138965,90.599455
5,Hernandez High School,District,4635,"$3,022,020.00",$652.00,3094,3748,66.752967,80.862999,53.527508
6,Holden High School,Charter,427,"$248,087.00",$581.00,395,411,92.505855,96.252927,89.227166
7,Huang High School,District,2917,"$1,910,635.00",$655.00,1916,2372,65.683922,81.316421,53.513884
8,Johnson High School,District,4761,"$3,094,650.00",$650.00,3145,3867,66.057551,81.222432,53.539172
9,Pena High School,Charter,962,"$585,858.00",$609.00,910,923,94.594595,95.945946,90.540541


In [ ]:
#HIGHEST-PERFORMING SCHOOLS

In [279]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = per_school_summary.sort_values(by='% Overall Passing', ascending=False)
top_schools.head(5)

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
1,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,1749,1803,94.133477,97.039828,91.334769
12,Thomas High School,Charter,1635,"$1,043,130.00",$638.00,1525,1591,93.272171,97.308869,90.948012
4,Griffin High School,Charter,1468,"$917,500.00",$625.00,1371,1426,93.392371,97.138965,90.599455
13,Wilson High School,Charter,2283,"$1,319,574.00",$578.00,2143,2204,93.867718,96.539641,90.582567
9,Pena High School,Charter,962,"$585,858.00",$609.00,910,923,94.594595,95.945946,90.540541


In [280]:
#BOTTOM PERFORMING SCHOOLS

In [281]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = per_school_summary.sort_values(by='% Overall Passing', ascending=True)
bottom_schools.head(5)

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
10,Rodriguez High School,District,3999,"$2,547,363.00",$637.00,2654,3208,66.366592,80.220055,52.988247
2,Figueroa High School,District,2949,"$1,884,411.00",$639.00,1946,2381,65.988471,80.739234,53.204476
7,Huang High School,District,2917,"$1,910,635.00",$655.00,1916,2372,65.683922,81.316421,53.513884
5,Hernandez High School,District,4635,"$3,022,020.00",$652.00,3094,3748,66.752967,80.862999,53.527508
8,Johnson High School,District,4761,"$3,094,650.00",$650.00,3145,3867,66.057551,81.222432,53.539172


In [282]:
# MATH SCORES BY GRADE

In [302]:
# Use the code provided to separate the data by grade
ninth_graders = combined_df[(combined_df["grade"] == "9th")]
tenth_graders = combined_df[(combined_df["grade"] == "10th")]
eleventh_graders = combined_df[(combined_df["grade"] == "11th")]
twelfth_graders = combined_df[(combined_df["grade"] == "12th")]

# Group by `school_name` and take the mean of the `math_score` column for each.
ninth_grader_math_scores = ninth_graders.groupby('school_name')['math_score'].mean()
tenth_grader_math_scores = tenth_graders.groupby('school_name')['math_score'].mean()
eleventh_grader_math_scores = eleventh_graders.groupby('school_name')['math_score'].mean()
twelfth_grader_math_scores = twelfth_graders.groupby('school_name')['math_score'].mean()

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
grades_df = {"9th Grade": ninth_grader_math_scores, 
             "10th Grade": tenth_grader_math_scores, 
             "11th Grade": eleventh_grader_math_scores, 
             "12th Grade": twelfth_grader_math_scores}
math_scores_by_grade =pd.DataFrame(grades_df)

# Minor data wrangling
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade




,9th Grade,10th Grade,11th Grade,12th Grade
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


In [303]:
#READING SCORES BY GRADE

In [309]:
# Use the code provided to separate the data by grade
ninth_graders = combined_df[(combined_df["grade"] == "9th")]
tenth_graders = combined_df[(combined_df["grade"] == "10th")]
eleventh_graders = combined_df[(combined_df["grade"] == "11th")]
twelfth_graders = combined_df[(combined_df["grade"] == "12th")]

# Group by `school_name` and take the mean of the the `reading_score` column for each.
ninth_grader_reading_scores = ninth_graders.groupby('school_name')['reading_score'].mean()
tenth_grader_reading_scores = tenth_graders.groupby('school_name')['reading_score'].mean()
eleventh_grader_reading_scores = eleventh_graders.groupby('school_name')['reading_score'].mean()
twelfth_grader_reading_scores = twelfth_graders.groupby('school_name')['reading_score'].mean()

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_grades = {"9th": ninth_grader_reading_scores, 
             "10th": tenth_grader_reading_scores, 
             "11th": eleventh_grader_reading_scores, 
             "12th": twelfth_grader_reading_scores}
reading_scores_by_grade =pd.DataFrame(reading_grades)

# Minor data wrangling
#reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


In [310]:
#Scores by School Spending

In [343]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [344]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()

In [359]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df['Per Student Budget'] = school_spending_df['Per Student Budget'].replace("$", "").replace(",", "")
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df["Per Student Budget"], bins=spending_bins, labels=labels)


In [352]:
#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()

In [361]:
# Assemble into DataFrame
spending_summary = pd.DataFrame({'Average math score': spending_math_scores, 'Average reading score': spending_reading_scores, 
                                 '% Passing math': spending_passing_math, 
                                 '% passing reading': spending_passing_reading,
                                 '% overall passing': overall_passing_spending})

# Display results
spending_summary

,Average math score,Average reading score,% Passing math,% passing reading,% overall passing
Spending Ranges (Per Student),,,,,
<$585,1491.750000,1539.25,93.460096,96.610877,90.369459
$585-630,1813.000000,2028.50,87.133538,92.718205,81.418596
$630-645,1999.000000,2338.00,73.484209,84.391793,62.857656
$645-680,2718.333333,3329.00,66.164813,81.133951,53.526855


In [311]:
#Scores by School Size

In [364]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [365]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

per_school_summary["School Size"] = pd.cut(per_school_summary['Total Students'], bins=size_bins, labels=labels)

per_school_summary

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,School Size
0,Bailey High School,District,4976,"$3,124,928.00",$628.00,3318,4077,66.680064,81.933280,54.642283,Large (2000-5000)
1,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,1749,1803,94.133477,97.039828,91.334769,Medium (1000-2000)
2,Figueroa High School,District,2949,"$1,884,411.00",$639.00,1946,2381,65.988471,80.739234,53.204476,Large (2000-5000)
3,Ford High School,District,2739,"$1,763,916.00",$644.00,1871,2172,68.309602,79.299014,54.289887,Large (2000-5000)
4,Griffin High School,Charter,1468,"$917,500.00",$625.00,1371,1426,93.392371,97.138965,90.599455,Medium (1000-2000)
5,Hernandez High School,District,4635,"$3,022,020.00",$652.00,3094,3748,66.752967,80.862999,53.527508,Large (2000-5000)
6,Holden High School,Charter,427,"$248,087.00",$581.00,395,411,92.505855,96.252927,89.227166,Small (<1000)
7,Huang High School,District,2917,"$1,910,635.00",$655.00,1916,2372,65.683922,81.316421,53.513884,Large (2000-5000)
8,Johnson High School,District,4761,"$3,094,650.00",$650.00,3145,3867,66.057551,81.222432,53.539172,Large (2000-5000)
9,Pena High School,Charter,962,"$585,858.00",$609.00,910,923,94.594595,95.945946,90.540541,Small (<1000)


In [366]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["School Size"])["Average Math Score"].mean()
size_reading_scores = per_school_summary.groupby(["School Size"])["Average Reading Score"].mean()
size_passing_math = per_school_summary.groupby(["School Size"])["% Passing Math"].mean()
size_passing_reading = per_school_summary.groupby(["School Size"])["% Passing Reading"].mean()
size_overall_passing = per_school_summary.groupby(["School Size"])["% Overall Passing"].mean()

In [367]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = pd.DataFrame({'Average math score': size_math_scores, 'Average reading score': size_reading_scores, 
                                 '% Passing math': size_passing_math, 
                                 '% passing reading': size_passing_reading,
                                 '% overall passing': size_overall_passing})

# Display results
size_summary

,Average math score,Average reading score,% Passing math,% passing reading,% overall passing
School Size,,,,,
Small (<1000),652.500,667.000,93.550225,96.099437,89.883853
Medium (1000-2000),1595.600,1649.400,93.599695,96.790680,90.621535
Large (2000-5000),2510.875,3003.625,69.963361,82.766634,58.286003


In [312]:
#Scores by School Type

In [368]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
average_math_score_by_type = per_school_summary.groupby(["School Type"])["Average Math Score"].mean()
average_reading_score_by_type = per_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
average_percent_passing_math_by_type = per_school_summary.groupby(["School Type"])["% Passing Math"].mean()
average_percent_passing_reading_by_type = per_school_summary.groupby(["School Type"])["% Passing Reading"].mean()
average_percent_overall_passing_by_type = per_school_summary.groupby(["School Type"])["% Overall Passing"].mean()

In [372]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = pd.DataFrame({'Average math score': average_math_score_by_type, 'Average reading score': average_reading_score_by_type, 
                                 '% Passing math': average_percent_passing_math_by_type, 
                                 '% passing reading': average_percent_passing_reading_by_type,
                                 '% overall passing': average_percent_overall_passing_by_type})

# Display results
type_summary

,Average math score,Average reading score,% Passing math,% passing reading,% overall passing
School Type,,,,,
Charter,1428.250000,1473.125000,93.620830,96.586489,90.432244
District,2563.428571,3117.857143,66.548453,80.799062,53.672208
